In [2]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Series
from predict_federal_reserve_interest_rates.util.date import parse_cpi_index_period
from predict_federal_reserve_interest_rates.util.viz import line_plot

In [3]:
# Importdata
# cpi_index_df = pd.read_csv("predict_federal_reserve_interest_rates/data/bls/cpi/bls_cpi_overall.txt",
#                            dtype = {'series_id': str,
#                                     'year': int,
#                                     'period': str,
#                                     'value': float,
#                                     'footnote_codes': str},
#                            delim_whitespace=True)
# cpi_index_df.rename(columns={"value": "rate"})

In [4]:
# Import data
cpi_index = pd.read_csv("data/bls/cpi/bls_cpi_current.txt", delim_whitespace=True)

In [5]:
cpi_index.head()

,series_id,year,period,value,footnote_codes
0,APU0000701111,1995,M01,0.238,NaN
1,APU0000701111,1995,M02,0.242,NaN
2,APU0000701111,1995,M03,0.242,NaN
3,APU0000701111,1995,M04,0.236,NaN
4,APU0000701111,1995,M05,0.244,NaN


In [6]:
cpi_index.mean()

/var/folders/3n/7kvkbqjd0l9grdj7kptkcfgr0000gn/T/ipykernel_21374/2130061905.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cpi_index.mean()


year    2007.653449
dtype: float64

In [135]:
# Clean up values for various items in the CPI
cpi_index['value'] = cpi_index['value'].replace(['-'], 0.00)
cpi_index = cpi_index.astype({'value': 'float64'})

In [136]:
cpi_index.iloc[[10667]]

,series_id,year,period,value,footnote_codes
10667,APU0000710211,2018,M11,0.0,NaN


In [137]:
cpi_index.shape

(190076, 5)

In [138]:
def get_period(row: Series):
    return parse_cpi_index_period(row["year"], row["period"])

In [139]:
cpi_index['period'] = cpi_index.apply(lambda row: get_period(row), axis=1)

In [140]:
cpi_index.dtypes

series_id            object
year                  int64
period            period[M]
value               float64
footnote_codes       object
dtype: object

In [141]:
cpi_index.head()

,series_id,year,period,value,footnote_codes
0,APU0000701111,1995,1995-01,0.238,NaN
1,APU0000701111,1995,1995-02,0.242,NaN
2,APU0000701111,1995,1995-03,0.242,NaN
3,APU0000701111,1995,1995-04,0.236,NaN
4,APU0000701111,1995,1995-05,0.244,NaN


In [142]:
cpi_index = cpi_index.set_index('period')

In [143]:
cpi_index.index

PeriodIndex(['1995-01', '1995-02', '1995-03', '1995-04', '1995-05', '1995-06',
             '1995-07', '1995-08', '1995-09', '1995-10',
             ...
             '2021-11', '2021-12', '2022-01', '2022-02', '2022-03', '2022-04',
             '2022-05', '2022-06', '2022-07', '2022-08'],
            dtype='period[M]', name='period', length=190076)

In [144]:
cpi_index.tail()

,series_id,year,value,footnote_codes
period,,,,
2022-04,APUS49G7471A,2022,4.733,NaN
2022-05,APUS49G7471A,2022,5.035,NaN
2022-06,APUS49G7471A,2022,5.582,NaN
2022-07,APUS49G7471A,2022,5.451,NaN
2022-08,APUS49G7471A,2022,4.976,NaN


In [103]:
len(cpi_index[cpi_index['year'] == 2022])

4131

In [109]:
len(cpi_index.loc[cpi_index.index == '2022'])

515

In [49]:
cpi_index.dtypes

series_id          object
year                int64
value             float64
footnote_codes     object
dtype: object

In [50]:
cpi_index.head()

,series_id,year,value,footnote_codes
period,,,,
1995-01,APU0000701111,1995,0.238,NaN
1995-02,APU0000701111,1995,0.242,NaN
1995-03,APU0000701111,1995,0.242,NaN
1995-04,APU0000701111,1995,0.236,NaN
1995-05,APU0000701111,1995,0.244,NaN


In [51]:
cpi_index[(cpi_index.value == 0.00)]

,series_id,year,value,footnote_codes
period,,,,
2018-11,APU0000710211,2018,0.0,NaN
2018-11,APU0200710211,2018,0.0,NaN
2018-11,APU030072511,2018,0.0,NaN
2018-12,APU030072511,2018,0.0,NaN
2018-11,APU035072511,2018,0.0,NaN
2018-12,APU035072511,2018,0.0,NaN
2018-11,APUS30072511,2018,0.0,NaN
2018-12,APUS30072511,2018,0.0,NaN
1996-03,APUS49B72601,1996,0.0,NaN


In [52]:
federal_funds_rates = pd.read_csv("data/fed/federal_funds_effective_rate.csv")

In [53]:
federal_funds_rates.head()

,time_period,federal_funds_effective_rate
0,1954-07-01,1.13
1,1954-07-02,1.25
2,1954-07-03,1.25
3,1954-07-04,1.25
4,1954-07-05,0.88


In [54]:
federal_funds_rates.shape

(24911, 2)

In [55]:
federal_funds_rates = federal_funds_rates.astype({'federal_funds_effective_rate': 'float64',
                                                      'time_period': 'datetime64'}) \
                                                .rename(columns={'time_period': 'period'}) \
                                                .set_index('period')

In [56]:
federal_funds_rates.index.dtype

dtype('<M8[ns]')

In [57]:
federal_funds_rates = federal_funds_rates.resample('M').mean()

In [58]:
federal_funds_rates.index.dtype

dtype('<M8[ns]')

In [59]:
federal_funds_rates.index = federal_funds_rates.index.to_period('M')

In [60]:
federal_funds_rates.index.dtype

period[M]

In [61]:
federal_funds_rates.head()

,federal_funds_effective_rate
period,
1954-07,0.799355
1954-08,1.220645
1954-09,1.066667
1954-10,0.848710
1954-11,0.833667


In [62]:
federal_funds_rates.shape

(819, 1)

In [63]:
cpi_index.shape

(190076, 4)

In [64]:
cpi_index.head()

,series_id,year,value,footnote_codes
period,,,,
1995-01,APU0000701111,1995,0.238,NaN
1995-02,APU0000701111,1995,0.242,NaN
1995-03,APU0000701111,1995,0.242,NaN
1995-04,APU0000701111,1995,0.236,NaN
1995-05,APU0000701111,1995,0.244,NaN


In [65]:
treasury_long_term_bond_rates = pd.read_csv("data/treasury/real-long-term-rates-2000-2021.csv")

In [66]:
treasury_long_term_bond_rates.head()

,date,long_term_real_average
0,12/31/21,-0.52
1,12/30/21,-0.51
2,12/29/21,-0.45
3,12/28/21,-0.48
4,12/27/21,-0.50


In [67]:
treasury_long_term_bond_rates = treasury_long_term_bond_rates.astype({'long_term_real_average': 'float64',
                                                                          'date': 'datetime64'}) \
                                                                    .rename(columns={'date': 'period'}) \
                                                                    .set_index('period')

In [68]:
treasury_long_term_bond_rates.dtypes

long_term_real_average    float64
dtype: object

In [69]:
treasury_long_term_bond_rate_monthly_average = treasury_long_term_bond_rates.rolling(window=30)['long_term_real_average'].mean()

In [70]:
treasury_long_term_bond_rates.shape

(5504, 1)

In [71]:
treasury_long_term_bond_rate_monthly_average.index = treasury_long_term_bond_rate_monthly_average.index.to_period('M')

In [113]:
treasury_long_term_bond_rates.dtypes

long_term_real_average    float64
dtype: object

In [73]:
treasury_long_term_bond_rates.head()

,long_term_real_average
period,
2021-12-31,-0.52
2021-12-30,-0.51
2021-12-29,-0.45
2021-12-28,-0.48
2021-12-27,-0.50


In [74]:
s_and_p_500_stock_prices = pd.read_csv("data/wsj/s_and_p_500_historical_prices.csv")

In [75]:
s_and_p_500_stock_prices = s_and_p_500_stock_prices.astype({'Date': 'datetime64'}) \
                                                                 .rename(columns={'Date': 'period'}) \
                                                                 .set_index('period')

In [76]:
s_and_p_500_stock_prices.shape

(11274, 4)

In [77]:
s_and_p_500_stock_prices = s_and_p_500_stock_prices.resample('M').mean()

In [78]:
s_and_p_500_stock_prices.shape

(537, 4)

In [79]:
s_and_p_500_stock_prices.dtypes

 Open     float64
 High     float64
 Low      float64
 Close    float64
dtype: object

In [80]:
bank_prime_rates = pd.read_csv("data/fed/bank_prime_rate.csv")

In [81]:
bank_prime_rates.head()

,time_period,bank_prime_rate
0,1955-08-04,3.25
1,1955-08-05,3.25
2,1955-08-06,3.25
3,1955-08-07,3.25
4,1955-08-08,3.25


In [82]:
s_and_p_500_stock_prices.head()

,Open,High,Low,Close
period,,,,
1978-01-31,90.254286,91.080000,89.747619,90.254286
1978-02-28,88.922632,89.453684,88.481053,88.922632
1978-03-31,88.819545,89.307727,88.189091,88.819545
1978-04-30,92.710500,93.397500,91.872500,92.710500
1978-05-31,97.412273,98.222273,96.579545,97.412273


In [83]:
cpi_index.head()

,series_id,year,value,footnote_codes
period,,,,
1995-01,APU0000701111,1995,0.238,NaN
1995-02,APU0000701111,1995,0.242,NaN
1995-03,APU0000701111,1995,0.242,NaN
1995-04,APU0000701111,1995,0.236,NaN
1995-05,APU0000701111,1995,0.244,NaN


In [84]:
treasury_long_term_bond_rates.head()

,long_term_real_average
period,
2021-12-31,-0.52
2021-12-30,-0.51
2021-12-29,-0.45
2021-12-28,-0.48
2021-12-27,-0.50


In [85]:
federal_funds_rates.head()

,federal_funds_effective_rate
period,
1954-07,0.799355
1954-08,1.220645
1954-09,1.066667
1954-10,0.848710
1954-11,0.833667


In [209]:
federal_funds_rates.head()

,federal_funds_effective_rate
period,
1954-07,0.799355
1954-08,1.220645
1954-09,1.066667
1954-10,0.848710
1954-11,0.833667


In [ ]:
cpi_index = cpi_index.groupby("period").mean()

In [170]:
federal_funds_rates_array = federal_funds_rates.loc[federal_funds_rates.index >= '2000'].index.to_numpy()

In [173]:
cpi_index_array = cpi_index.loc[cpi_index.index >= '2000'].index.to_numpy()

In [ ]:
series = zip(cpi_index_array, federal_funds_rates_array)

In [210]:
federal_funds_rates_after_2000 = federal_funds_rates.loc[federal_funds_rates.index >= '2000']

In [211]:
cpi_index_after_2000 = cpi_index.loc[cpi_index.index >= '2000']

In [218]:
federal_funds_rates_array_after_2000 = federal_funds_rates_after_2000["federal_funds_effective_rate"].to_numpy()

In [222]:
cpi_index_array_after_2000 = cpi_index_after_2000["value"].to_numpy()

In [223]:
len(cpi_index_array_after_2000)

272

In [224]:
series = zip(cpi_index_array_after_2000, federal_funds_rates_array_after_2000)

In [226]:
cpi_index_array_after_2000

array([ 9.52256979,  9.58644118,  9.63173607,  9.71011148,  9.96946477,
       10.31310116, 10.57736469, 10.48265455, 10.94900493, 11.36254257,
       11.61160745, 12.07371986, 13.23714165, 13.38721477, 13.13539226,
       12.92385835, 12.75986449, 12.29582624, 11.81923729, 11.4629661 ,
       11.19101356, 10.74599324, 10.90429032, 11.07567007, 10.9928277 ,
       10.55014622, 10.53710235, 10.74822635, 10.82535204, 10.82369165,
       10.84518228, 10.84081122, 10.90891823, 11.06527474, 11.37029503,
       11.4358431 , 11.7274244 , 12.01072432, 13.02301541, 12.82549742,
       12.75430314, 12.93742957, 12.88111938, 12.81959619, 12.88040728,
       12.77716118, 12.88154974, 13.03076923, 13.2375    , 13.12379758,
       12.84624525, 12.88053356, 13.25533042, 13.68098084, 13.83050877,
       13.80781739, 13.48255095, 13.88673545, 14.68927208, 14.76189735,
       14.63040845, 14.15604878, 14.10347644, 14.71133625, 14.84603186,
       14.60176532, 14.93841739, 15.15142683, 16.41889043, 18.31

In [227]:
federal_funds_rates_array_after_2000

array([5.4483871 , 5.73482759, 5.85354839, 6.01966667, 6.26806452,
       6.52833333, 6.54451613, 6.49677419, 6.517     , 6.50935484,
       6.51433333, 6.40225806, 5.97580645, 5.49428571, 5.31096774,
       4.802     , 4.21225806, 3.96966667, 3.76903226, 3.65258065,
       3.069     , 2.48516129, 2.087     , 1.81774194, 1.72967742,
       1.73964286, 1.73      , 1.753     , 1.75129032, 1.75033333,
       1.72903226, 1.74322581, 1.75066667, 1.75483871, 1.336     ,
       1.2383871 , 1.23516129, 1.26214286, 1.25290323, 1.258     ,
       1.25903226, 1.22266667, 1.01129032, 1.02903226, 1.01      ,
       1.01      , 0.996     , 0.98419355, 0.99709677, 1.00724138,
       1.0016129 , 1.004     , 1.00451613, 1.02566667, 1.26354839,
       1.42935484, 1.605     , 1.76096774, 1.932     , 2.15612903,
       2.27935484, 2.50178571, 2.62903226, 2.785     , 3.00258065,
       3.03566667, 3.26258065, 3.49967742, 3.623     , 3.77935484,
       3.999     , 4.15709677, 4.28516129, 4.49142857, 4.58935

In [1]:
count = 1
for x, y in series:
    print(f"{x}, {y}")
    count +=1
    
print(count)

NameError: name 'series' is not defined